In [1]:
import os
import sys
import boto
import ast
from datetime import datetime
HOME = os.environ.get('HOME')
sys.path.append(HOME + "/comscore_processing")
from utils.utils import apply_process_row, get_log_date,process_row
from utils.log_processors import UserProcessor
#from RDLogging.logging import RDLogger
sc.addPyFile(HOME + "/comscore_processing/utils.zip")
sc.addPyFile(HOME + "/comscore_processing/RDLogging.zip")
from utils.constants import *
from utils.utils import get_date_range,eval_dict
from utils.session_aggregators import SessionAggregator
from utils.dynamo_writer import record_log_date
from utils.category_counts import process_category_combos
from boto.dynamodb2.layer1 import DynamoDBConnection
from utils.user_ids_by_site import write_user_ids_by_site
from utils.parquet_writer import ParquetWriter,VISITOR_SCHEMA
from boto.s3.connection import S3Connection
from boto.s3.connection import OrdinaryCallingFormat
from operator import add
from utils.category_counts import site_combos_to_process
import csv, io
from urllib.parse import urlparse,parse_qs
import argparse
from utils.config import config
from utils.utils import (
        S3_FIRST_PASS_PATTERN, del_s3_dir, RDMSR_BUCKET,
        get_date_range, parser, eval_dict,
        combineLists,period_delta)

In [2]:
sa = SessionAggregator(datetime(2016, 5, 23), 
        datetime(2016, 7, 3))
from inspect import getmembers
header = ['visitor_id', 'site']
for member in getmembers(sa):
    if callable(member[1]) and not member[0].startswith('_'):
        header.append(member[0])
header = sc.broadcast(header)
header.value

['visitor_id',
 'site',
 'abandon_indicator',
 'afternoon_visit_indicator',
 'average_page_views_per_visit',
 'average_visit_time_per_category',
 'average_visit_time_per_day',
 'average_visit_time_spent',
 'bing_referral',
 'browser',
 'category_terms',
 'closed_hc_popup',
 'content_terms',
 'date_of_last_order',
 'date_of_last_payment_event',
 'date_of_last_visit',
 'days_since_last_abandon',
 'days_since_last_customer_support_visit',
 'ehid',
 'email',
 'epid',
 'evening_visit_indicator',
 'external_referrer_by_day',
 'facebook_content_terms',
 'facebook_content_terms_shared',
 'facebook_referral',
 'first_external_referrer_domain_to_site',
 'first_internal_referrer_to_site',
 'google_content_terms_shared',
 'google_plus_content_terms',
 'google_referrel',
 'healthcare_conditions_selected',
 'ip_address',
 'ip_addresses',
 'is_app_installed',
 'is_bot',
 'is_mobile',
 'list_of_newsletters',
 'lunch_visit_indicator',
 'mean_time_between_visits',
 'median_time_of_day_of_visits',
 'morn

In [3]:
s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-05-23",end="2016-07-03",source="parquet"))
#s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-06-11",end="2016-06-12",source="parquet"))
def getMappedData(header):
    def innerGetMappedData(row):
        splittedRow = row.split('\t')
        row = dict(zip(header.value[:len(splittedRow)], splittedRow))
        return row
    return innerGetMappedData

def mapData(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData(header)).filter(lambda x: x is not None)
    return vData
visitFinalData = mapData(s3_visitor_data)
### VFD filter, submitted_hc_popup is date on which the user has submitted healthcare popup. Blank otherwise
VFD=visitFinalData.filter(lambda x: x['submitted_hc_popup']!='')

#VFD.coalesce(1).saveAsTextFile(path='s3n://rdmsr-anl/test_data/2016-05-24_to_2016-07-03_HC_Submitted_final_pass',\
#                         compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")
#VFDsample = VFD.sample(False,0.4)
#VFD.saveAsTextFile(path='/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_NOT_Submitted_final_pass',\
#                        compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [5]:
VFD.take(1)

[{'abandon_indicator': '0',
  'afternoon_visit_indicator': '1',
  'average_page_views_per_visit': '1.0',
  'average_visit_time_per_category': 'health~healthy eating:1096',
  'average_visit_time_per_day': '2016-06-23:1117',
  'average_visit_time_spent': '1117.0',
  'bing_referral': '0',
  'browser': 'Chrome',
  'category_terms': 'health~healthy eating:2:1096',
  'closed_hc_popup': '2016-06-23',
  'content_terms': ':1:1112,antioxidant:1:1112,best:1:1112,eating:1:1112,foods:1:1112,health:1:1112,healthy:1:1112,rich:1:1112',
  'date_of_last_order': 'no orders',
  'date_of_last_payment_event': 'no payments',
  'date_of_last_visit': '2016-06-23',
  'days_since_last_abandon': '0',
  'days_since_last_customer_support_visit': '',
  'ehid': 'no ehid',
  'email': 'no email',
  'epid': 'no epid',
  'evening_visit_indicator': '1',
  'external_referrer_by_day': '2016-06-23:www.google.co.in/www.google.co.in',
  'facebook_content_terms': '',
  'facebook_content_terms_shared': '',
  'facebook_referral':

In [26]:
### get the Keys from the Visitor Final Data, what's it for???
def getkeys(row):
    x = row.items()
    vals_list = []
    for k,v in x:
        vals_list.append('"'+k+'"')
    return ','.join(vals_list)
vals = VFD.map(getkeys)
### type(vals.take(1)[0]) == str, why not just output the header into a str?
'''
res = vals.take(1)[0]
for key in header.value:
    print(key in res)
'''
### will all return TRUE

dict

In [31]:
''' john: 
This function return a list of values for each row in visitor final data.
it set NULL for empty value;
it also reorder and reformated the value for tag 'healthcare_conditions_selected';
'''
def getValues(row):
    x = row.items()
    vals_list = []
    for k,v in x: ### loop each row with k:key and v:value pair.
        if v:
            if k == 'healthcare_conditions_selected':
                conditions_list = ['Diabetes', 'COPD', 'High Cholesterol', 'Heart Disease', 'Allergy', 'Other Ailments']
                selected_list = v.split(',')
                prepared_conditions_list = []
                if 'Diabetes' in selected_list:
                    prepared_conditions_list.append('Diabetes')
                else:
                    prepared_conditions_list.append('Null')
                if 'COPD' in selected_list:
                    prepared_conditions_list.append('COPD')
                else:
                    prepared_conditions_list.append('Null')
                if 'High Cholesterol' in selected_list:
                    prepared_conditions_list.append('High Cholesterol')
                else:
                    prepared_conditions_list.append('Null')
                if 'Heart Disease' in selected_list:
                    prepared_conditions_list.append('Heart Disease')
                else:
                    prepared_conditions_list.append('Null')
                if 'Allergy' in selected_list:
                    prepared_conditions_list.append('Allergy')
                else:
                    prepared_conditions_list.append('Null')
                if 'Other Ailments' in selected_list:
                    prepared_conditions_list.append('Other Ailments')
                else:
                    prepared_conditions_list.append('Null')
                v = ','.join(prepared_conditions_list)
            vals_list.append('"'+v+'"')
        else: vals_list.append('"'+"NULL"+'"')
    return ','.join(vals_list)

vals = VFD.map(getValues)
vals.take(5)
#vals_sample = vals.sample(False,0.001)
#vals_sample.count()
vals.coalesce(1).\
            saveAsTextFile('/home/analytics/testdata/hc_popup_fnlp_submitted')

In [ ]:
hc_not_submitted_users_sample.take(10)

In [ ]:
def getValues(row):
    x = row.items()
    vals_list = []
    for k,v in x:
        if k!='session_times':
            vals_list.append(v)
    return ','.join(vals_list)

vals = VFD.map(getValues)
hc_not_submitted_users_sample = vals.sample(False,0.001)
hc_not_submitted_users_sample.coalesce(1).\
            saveAsTextFile('/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_NOT_Sub_final_pass_sample_values_no_datetime')

In [ ]:
hc_submitted_users_sample.coalesce(1).\
            saveAsTextFile('/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_Sub_final_pass_sample_values')

In [ ]:
hc_not_submitted_users_sample.coalesce(1).\
            saveAsTextFile(path='/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_Submitted_final_pass_sample_2000_values',\
                        compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
hc_submitted_users = sc.textFile('s3n://rdmsr-anl/test_data/2016-05-24_to_2016-07-03_HC_Submitted_final_pass')
#hc_submitted_users = sc.textFile('s3n://rdmsr-anl/test_data/2016-05-22_to_2016-07-03_HC_Submitted')
hc_submitted_users.count()

In [ ]:
hc_submitted_users = sc.textFile('s3n://rdmsr-anl/test_data/2016-05-24_to_2016-07-03_HC_Submitted_final_pass')
sample = hc_submitted_users.sample(False,0.2)
sample.coalesce(1).saveAsTextFile(path='/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_Submitted_final_pass_sample_2000',\
                        compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
hc_not_submitted_users = sc.textFile('/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_NOT_Submitted_final_pass')
sample = hc_not_submitted_users.sample(False,0.001)
sample.coalesce(1).saveAsTextFile(path='/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_NOT_Submitted_final_pass_sample_2000',\
                        compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
hc_not_submitted_users = sc.textFile('/home/analytics/testdata/2016-05-24_to_2016-07-03_HC_NOT_Submitted_final_pass_sample_2000')
def getValues(row):
    row = ast.literal_eval(row)
    return row

vals = hc_not_submitted_users.map(getValues)
vals.take(1)

In [ ]:
def get_average_visit_time_per_day(row):
    row = ast.literal_eval(row)
    if row['average_visit_time_per_day']:
        dates = row['average_visit_time_per_day'].split(',')
        for date1 in dates:
            if len(date1.split(':'))>1:
                time_str = date1.split(':')[1]
                time_spent = int(time_str)/60
                if time_spent < 5:
                    return 5
                elif 5 <= time_spent < 10:
                    return 10
                elif 10 <= time_spent < 15:
                    return 15
                else:
                    return 20
hcdates = hc_submitted_users.map(get_average_visit_time_per_day).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_average_page_views_per_visit(row):
    row = ast.literal_eval(row)

    if row['average_page_views_per_visit'] != 'average_page_views_per_visit':
        avg = float(row['average_page_views_per_visit'])
        if avg < 5:
            return 5
        elif 5 <= avg < 10:
            return 10
        elif 10 <= avg < 15:
            return 15
        else:
            return 20
hcdates = hc_submitted_users.map(get_average_page_views_per_visit).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_survey_taker_indicator(row):
    row = ast.literal_eval(row)
    return row['survey_taker_indicator'] if row['survey_taker_indicator'] else None
hcdates = hc_submitted_users.map(get_date_of_last_order).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_date_of_last_order(row):
    row = ast.literal_eval(row)
    return row['date_of_last_order'] if row['date_of_last_order']!='no orders' else None
hcdates = hc_submitted_users.map(get_date_of_last_order).filter(lambda x: x is not None)
hcdates.take(10)

In [ ]:
def get_most_frequest_date_of_visit(row):
    row = ast.literal_eval(row)
    if row['average_visit_time_per_day']:
        dates = row['average_visit_time_per_day'].split(',')
        for date1 in dates:
            if len(date1.split(':'))>1:
                date_v = date1.split(':')[0]
                return datetime.strptime(date_v,'%Y-%m-%d').strftime('%A')
hcdates = hc_submitted_users.map(get_most_frequest_date_of_visit).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_most_frequent_referrer(row):
    row = ast.literal_eval(row)
    referrers = []
    #row['bing_referral']!='bing_referral' and row['bing_referral']!='0'
    #row['facebook_referral']!='facebook_referral' and row['facebook_referral']!='0'
    #row['pinterest_referral']!='pinterest_referral' and row['pinterest_referral']!='0'
    #row['twitter_referral']!='twitter_referral' and row['twitter_referral']!='0'
    #row['yahoo_referral']!='yahoo_referral' and row['yahoo_referral']!='0'
    referrers.append('bing') if (row['bing_referral'] and \
                 row['bing_referral']!='bing_referral' and row['bing_referral']!='0') else None
    referrers.append('facebook') if (row['facebook_referral'] and \
                 row['facebook_referral']!='facebook_referral' and row['facebook_referral']!='0') else None
    referrers.append('pinterest') if (row['pinterest_referral'] and \
                 row['pinterest_referral']!='pinterest_referral' and row['pinterest_referral']!='0') else None
    referrers.append('twitter') if (row['twitter_referral'] and \
                 row['twitter_referral']!='twitter_referral' and row['twitter_referral']!='0') else None
    referrers.append('yahoo') if (row['yahoo_referral'] and \
                 row['yahoo_referral']!='yahoo_referral' and row['yahoo_referral']!='0') else None
    referrers.append('google') if (row['google_referrel'] and \
                 row['google_referrel']!='google_referrel' and row['google_referrel']!='0') else None
    return referrers
hcdates = hc_submitted_users.flatMap(get_most_frequent_referrer).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_most_frequest_date_of_submission(row):
    row = ast.literal_eval(row)
    if row['submitted_hc_popup']:
        dates = row['submitted_hc_popup'].split(',')
        for date1 in dates:
            if date1!='submitted_hc_popup':
                return datetime.strptime(date1,'%Y-%m-%d').strftime('%A')
hcdates = hc_submitted_users.map(get_most_frequest_date_of_visit).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_most_time_spent_category_diabetes(row):
    row = ast.literal_eval(row)
    max_time = 0
    cat = None
    conditions_selected = row['healthcare_conditions_selected'].split(',')
    if 'Diabetes' in conditions_selected:
        if row['average_visit_time_per_category']:
            cats_times = row['average_visit_time_per_category'].split(',')
            for cat_time in cats_times:
                time = cat_time.split(':')[1]
                if time!=' microsite' and int(time) > max_time:
                    cat = cat_time.split(':')[0]
    return cat
hcdates = hc_submitted_users.map(get_most_time_spent_category_diabetes).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_date_of_last_order(row):
    row = ast.literal_eval(row)
    conditions = []
    return [c for c in row['healthcare_conditions_selected'].split(',') if row['healthcare_conditions_selected'] ]
hcdates = hc_submitted_users.flatMap(get_date_of_last_order).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_date_of_last_order(row):
    row = ast.literal_eval(row)
    conditions = []
    return [c for c in row['healthcare_conditions_selected'].split(',') if row['healthcare_conditions_selected'] ]
hcdates = hc_submitted_users.flatMap(get_date_of_last_order).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
def get_frequent_time_of_visit(row):
    row = ast.literal_eval(row)
    frequent_hour = []
    session_times = list(row['session_times'])
    for s_time in session_times:
        frequent_hour.append(s_time)
    return frequent_hour
hcdates = hc_submitted_users.map(get_frequent_time_of_visit).filter(lambda x: x is not None)
hcdates.take(10)